In [1]:
import cv2

import torch

from sort import *

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

mot_tracker = Sort(max_age=10)

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True);
model.float()
model.eval();

Using cache found in C:\Users\user/.cache\torch\hub\ultralytics_yolov5_master
WARNING  'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING  'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")

YOLOv5  2023-5-24 Python-3.9.18 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [4]:
test_video_path = './test_video/test5.mp4'

if os.path.isfile(test_video_path):	# 해당 파일이 있는지 확인
    vid = cv2.VideoCapture(test_video_path)

i = 0
while(True):
    ret, image_show = vid.read()
    image_show = cv2.resize(image_show, (500, 300))
    
    preds = model(image_show)
    detections = preds.pred[0].to('cpu').numpy()
    
    # Filtering for human 
    person_class = (detections[:, -1] == 0)
    detections = detections[person_class, :]
    track_bbs_ids = mot_tracker.update(detections)
    
    for j in range(len(track_bbs_ids.tolist())):
        
        coords = track_bbs_ids.tolist()[j]
        x1, y1, x2, y2 = int(coords[0]), int(coords[1]), int(coords[2]), int(coords[3])
        name_idx = int(coords[4])
        name = "ID : {}".format(str(name_idx))
        color = (255, 255, 0)
        
        cv2.rectangle(image_show, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image_show, name, (x2, y2+10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        cv2.imshow('image', image_show)
        
    cv2.imshow('image', image_show)
    
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break


vid.release()
cv2.destroyAllWindows()